In [1]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
from canopy.tokenizer import Tokenizer
from canopy.knowledge_base import KnowledgeBase
from canopy.models.data_models import Query
from canopy.context_engine import ContextEngine
from canopy.models.data_models import UserMessage,AssistantMessage
import gradio as gr

In [2]:
load_dotenv(find_dotenv())
gradio_port=os.getenv("PORT1")
index_name = "pinecone-docs"
Tokenizer.initialize()

kb = KnowledgeBase(index_name)
context_engine = ContextEngine(kb)
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)
kb.connect()

In [3]:
def gradio_to_canopy(gradio_chatbot,message):
    canopy_list=[]
    last_message=UserMessage(content=message)
    for turn in gradio_chatbot:
        user_message, bot_message = turn
        canopy_list.append(UserMessage(content=user_message))
        canopy_list.append(AssistantMessage(content=bot_message))
    canopy_list.append(last_message)
    return canopy_list

def respond(message, gradio_chatbot):
        canopy_chat_history=gradio_to_canopy(gradio_chatbot,message)
        chat_response=chat_engine.chat(canopy_chat_history)
        bot_response=chat_response.choices[0].message #This is a canopy object
        gradio_chatbot.append((message, bot_response.content))
        return "", gradio_chatbot



In [4]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240,show_copy_button=True) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Enviar")
    clear = gr.ClearButton(components=[msg, chatbot], value="Limpiar")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT1']))

Running on local URL:  http://127.0.0.1:8999
Running on public URL: https://517683abf67f0da357.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
